In [1]:
# Install the nvcc4jupyter plugin
!pip install nvcc4jupyter

# Load the extension into the notebook
%load_ext nvcc4jupyter


Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpsra69gdy".


In [2]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

int main() {
    int nDevices;
    cudaGetDeviceCount(&nDevices);
    for (int i = 0; i < nDevices; i++) {
        cudaDeviceProp prop;
        cudaGetDeviceProperties(&prop, i);
        printf("Device Number: %d\n", i);
        printf("  Device name: %s\n", prop.name);
        printf("  Memory Clock Rate (KHz): %d\n", prop.memoryClockRate);
        printf("  Memory Bus Width (bits): %d\n", prop.memoryBusWidth);
        printf("  Peak Memory Bandwidth (GB/s): %f\n\n",
               2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);
    }
    return 0;
}


Device Number: 0
  Device name: Tesla T4
  Memory Clock Rate (KHz): 5001000
  Memory Bus Width (bits): 256
  Peak Memory Bandwidth (GB/s): 320.064000




In [8]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void kernel_1t1e(float *A, float *B, float *C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int index = row * N + col;

    if (row < N && col < N) {
        A[index] = B[index] + C[index];
    }
}

void randomInit(float* data, int size) {
    for (int i = 0; i < size; i++)
        data[i] = rand() / (float)RAND_MAX * 100.0;
}

int main() {
    int sizes[] = {4, 16, 64, 256, 1024, 4096, 16384}; // Matrix sizes to test
    int numSizes = sizeof(sizes) / sizeof(sizes[0]);
    int numRuns = 10;

    for (int s = 0; s < numSizes; s++) {
        int N = sizes[s];
        size_t size = N * N * sizeof(float);
        float *A, *B, *C, *d_A, *d_B, *d_C;
        float totalMilliseconds = 0.0, avgMilliseconds;

        // Allocate space for host copies of A, B, C and setup values
        A = (float *)malloc(size); randomInit(A, N*N);
        B = (float *)malloc(size); randomInit(B, N*N);
        C = (float *)malloc(size); randomInit(C, N*N);

        // Allocate space for device copies of A, B, C
        cudaMalloc((void **)&d_A, size);
        cudaMalloc((void **)&d_B, size);
        cudaMalloc((void **)&d_C, size);

        // Copy inputs to device
        cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);

        // Setup CUDA events for timing
        cudaEvent_t start, end;
        cudaEventCreate(&start);
        cudaEventCreate(&end);

        for (int i = 0; i < numRuns; i++) {
            // Record event and launch kernel
            cudaEventRecord(start);

            // Launch kernel_1t1c() kernel on GPU
            dim3 threadsPerBlock(1);
            dim3 numBlocks(N);
            kernel_1t1e<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, N);

            // Stop recording
            cudaEventRecord(end);
            cudaEventSynchronize(end); // Wait for the stop event to complete

            float milliseconds = 0;

            // Calculate elapsed time
            cudaEventElapsedTime(&milliseconds, start, end);
            totalMilliseconds += milliseconds;
        }

        avgMilliseconds = totalMilliseconds / numRuns;
        printf("N = %d: Average Time for kernel_1t1c over %d runs: %f ms\n", N, numRuns, avgMilliseconds);

        // Wait for GPU to finish before accessing on host
        cudaDeviceSynchronize();

        // Copy result back to host
        cudaMemcpy(A, d_A, size, cudaMemcpyDeviceToHost);

        // Print results or check output
        printf("First 10 elements for N = %d:\n", N);
        for (int i = 0; i < 10 && i < N; i++) { // Example of printing first 10 elements
            printf("%f ", A[i]);
        }
        printf("\n");

        // Cleanup
        cudaEventDestroy(start);
        cudaEventDestroy(end);
        cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
        free(A); free(B); free(C);
    }

    return 0;
}



N = 4: Average Time for kernel_1t1c over 10 runs: 0.023427 ms
First 10 elements for N = 4:
124.835159 101.332863 77.915482 113.125603 
N = 16: Average Time for kernel_1t1c over 10 runs: 0.006826 ms
First 10 elements for N = 16:
74.517647 98.296608 18.021336 99.150787 143.830017 130.534576 104.634445 129.568481 97.333878 128.621811 
N = 64: Average Time for kernel_1t1c over 10 runs: 0.006458 ms
First 10 elements for N = 64:
51.953617 89.961128 61.007092 38.918518 170.510712 138.459351 109.247101 99.387833 164.447388 99.482315 
N = 256: Average Time for kernel_1t1c over 10 runs: 0.007344 ms
First 10 elements for N = 256:
40.772362 65.722710 128.192261 50.025650 61.464783 177.513412 120.943428 73.261261 154.609848 95.283554 
N = 1024: Average Time for kernel_1t1c over 10 runs: 0.009325 ms
First 10 elements for N = 1024:
136.829529 122.516586 98.160301 84.390724 27.842970 93.901207 71.524582 172.090012 108.210281 102.350830 
N = 4096: Average Time for kernel_1t1c over 10 runs: 0.017562 ms


# Analysis of each kernel

## `kernel_1t1e` (One Thread per Element)

This kernel is designed to perform addition on two matrices, where each thread in the grid has to add a single element from two input matrices B and C, and store the result in the corresponding element of the output matrix A. This design allows the kernel to fully utilize the GPU's parallel processing capabilities by assigning one thread to each operation, which is straightforward and maximizes parallelism at the element level.

### Key characteristics
* **Simple**: Each thread handles a single element, making the program logic straightforward.
* **Scalable**: This approach is robust enough to handle effectively and efficiently the size of the matrices given that there are enough GPU resources.

### Launch configuration

`kernel_1t1e` involves setting up a grid of threads such that there is a thread for each element of the corresponding matrix operands.

* **Threads per block**: These are 2-dimensional, and in all of the test runs to `kernel_1t1e`, this was set as 16x16, which fits in well with our constraint of maximum threads per block limit. This is well suited also for this kind of memory access pattern, as we will discuss below.
* **Blocks per grid**. This is calculated for covering the whole matrix, with the number of blocks in each grid dimension set as $\lceil N/threads \rceil$, where $N$ is the dimension of the matrix.

### Memory access pattern

The function `kernel_1t1e` benefits from coalesced memory access pattern when each thread access consecutive elements of the matrices. In a row-major storage order, if each thread access an element that lies consecutively in memory (e.g. elements of a row when accessed by threads in a warp), the memory transactions are coalesced into as few as possible, which maximizes memory throughput.

* **Coalesced Access**: When threads within the same warp access consecutive memory locations, the accesses can be coalesced into a single memory transaction, reducing memory latency and increasing bandwidth utilization.
* **Data Alignment**: Ensuring that data is aligned to 32-byte or 128-byte boundaries (depending on the compute capability of the GPU) can further enhance the efficiency of memory accesses.


In [7]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void kernel_1t1r(float *A, float *B, float *C, int N) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N) { // Ensure the row is within the matrix
        int index = row * N;
        for (int col = 0; col < N; col++) {
            A[index + col] = B[index + col] + C[index + col];
        }
    }
}

void randomInit(float* data, int size) {
    for (int i = 0; i < size; i++)
        data[i] = rand() / (float)RAND_MAX * 100.0;
}

int main() {
    int sizes[] = {4, 16, 64, 256, 1024, 4096, 16384}; // Matrix sizes to test
    int numSizes = sizeof(sizes) / sizeof(sizes[0]);
    int numRuns = 10;

    for (int s = 0; s < numSizes; s++) {
        int N = sizes[s];
        size_t size = N * N * sizeof(float);
        float *A, *B, *C, *d_A, *d_B, *d_C;
        float totalMilliseconds = 0.0, avgMilliseconds;

        // Allocate space for host copies of A, B, C and setup values
        A = (float *)malloc(size); randomInit(A, N*N);
        B = (float *)malloc(size); randomInit(B, N*N);
        C = (float *)malloc(size); randomInit(C, N*N);

        // Allocate space for device copies of A, B, C
        cudaMalloc((void **)&d_A, size);
        cudaMalloc((void **)&d_B, size);
        cudaMalloc((void **)&d_C, size);

        // Copy inputs to device
        cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);

        // Setup CUDA events for timing
        cudaEvent_t start, end;
        cudaEventCreate(&start);
        cudaEventCreate(&end);

        for (int i = 0; i < numRuns; i++) {
            // Record event and launch kernel
            cudaEventRecord(start);

            // Launch kernel_1t1c() kernel on GPU
            dim3 threadsPerBlock(1);
            dim3 numBlocks(N);
            kernel_1t1r<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, N);

            // Stop recording
            cudaEventRecord(end);
            cudaEventSynchronize(end); // Wait for the stop event to complete

            float milliseconds = 0;

            // Calculate elapsed time
            cudaEventElapsedTime(&milliseconds, start, end);
            totalMilliseconds += milliseconds;
        }

        avgMilliseconds = totalMilliseconds / numRuns;
        printf("N = %d: Average Time for kernel_1t1c over %d runs: %f ms\n", N, numRuns, avgMilliseconds);

        // Wait for GPU to finish before accessing on host
        cudaDeviceSynchronize();

        // Copy result back to host
        cudaMemcpy(A, d_A, size, cudaMemcpyDeviceToHost);

        // Print results or check output
        printf("First 10 elements for N = %d:\n", N);
        for (int i = 0; i < 10 && i < N; i++) { // Example of printing first 10 elements
            printf("%f ", A[i]);
        }
        printf("\n");

        // Cleanup
        cudaEventDestroy(start);
        cudaEventDestroy(end);
        cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
        free(A); free(B); free(C);
    }

    return 0;
}



N = 4: Average Time for kernel_1t1c over 10 runs: 0.032054 ms
First 10 elements for N = 4:
124.835159 101.332863 77.915482 113.125603 
N = 16: Average Time for kernel_1t1c over 10 runs: 0.008710 ms
First 10 elements for N = 16:
74.517647 98.296608 18.021336 99.150787 143.830017 130.534576 104.634445 129.568481 97.333878 128.621811 
N = 64: Average Time for kernel_1t1c over 10 runs: 0.014781 ms
First 10 elements for N = 64:
51.953617 89.961128 61.007092 38.918518 170.510712 138.459351 109.247101 99.387833 164.447388 99.482315 
N = 256: Average Time for kernel_1t1c over 10 runs: 0.040493 ms
First 10 elements for N = 256:
40.772362 65.722710 128.192261 50.025650 61.464783 177.513412 120.943428 73.261261 154.609848 95.283554 
N = 1024: Average Time for kernel_1t1c over 10 runs: 0.366890 ms
First 10 elements for N = 1024:
136.829529 122.516586 98.160301 84.390724 27.842970 93.901207 71.524582 172.090012 108.210281 102.350830 
N = 4096: Average Time for kernel_1t1c over 10 runs: 5.231472 ms


# Analysis of each kernel

## `kernel_1t1r` (One Thread per Row)

This configuration assigns one thread to handle the addition of all elements in a single row of matrices $B$ and $C$, storing the result in the corresponding row of output matrix $A$. Each thread computes an entire row, which means it iterates over all columns within that row to perform the addition.

### Key characteristics
* **Simplicity in control flow**: Each thread handles a single row, simplifying the control structure within the thread as it loops over columns.
* **Reduced thread utilization**: Fewer threads are activated compared to `kernel_1t1e`, as only one thread is needed per row, which might be beneficial for smaller matrices or specific GPU architectures.

### Launch configuration

`kernel_1t1r`'s launch configuration typically involves fewer blocks since each block can handle rows, if needed.

* **Threads per block**: Could be set to handle multiple rows per block depending on the matrix size, often kept at one thread per block if each thread handles an entire row.
* **Blocks per grid**. Equals the number of rows in the matrix, ensuring that each row is handled by one thread.

### Memory access pattern

The memory access pattern here is strided, as each thread accesses elements that are spaced apart by the length of the rows (i.e. the matrix's width).

* **Threads per block**: Threads access memory in a strided pattern, which can result in non-coalesced memory accesses, leading to inefficiencies.
* **Cache utilization**: Poor cache utilization due to the strided access pattern, which might increase memory latency.

In [6]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void kernel_1t1c(float *A, float *B, float *C, int N) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (col < N) { // Ensure the column is within the matrix
        for (int row = 0; row < N; row++) {
            int index = row * N + col;
            A[index] = B[index] + C[index];
        }
    }
}

void randomInit(float* data, int size) {
    for (int i = 0; i < size; i++)
        data[i] = rand() / (float)RAND_MAX * 100.0;
}

int main() {
    int sizes[] = {4, 16, 64, 256, 1024, 4096, 16384}; // Matrix sizes to test
    int numSizes = sizeof(sizes) / sizeof(sizes[0]);
    int numRuns = 10;

    for (int s = 0; s < numSizes; s++) {
        int N = sizes[s];
        size_t size = N * N * sizeof(float);
        float *A, *B, *C, *d_A, *d_B, *d_C;
        float totalMilliseconds = 0.0, avgMilliseconds;

        // Allocate space for host copies of A, B, C and setup values
        A = (float *)malloc(size); randomInit(A, N*N);
        B = (float *)malloc(size); randomInit(B, N*N);
        C = (float *)malloc(size); randomInit(C, N*N);

        // Allocate space for device copies of A, B, C
        cudaMalloc((void **)&d_A, size);
        cudaMalloc((void **)&d_B, size);
        cudaMalloc((void **)&d_C, size);

        // Copy inputs to device
        cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);

        // Setup CUDA events for timing
        cudaEvent_t start, end;
        cudaEventCreate(&start);
        cudaEventCreate(&end);

        for (int i = 0; i < numRuns; i++) {
            // Record event and launch kernel
            cudaEventRecord(start);

            // Launch kernel_1t1c() kernel on GPU
            dim3 threadsPerBlock(1);
            dim3 numBlocks(N);
            kernel_1t1c<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, N);

            // Stop recording
            cudaEventRecord(end);
            cudaEventSynchronize(end); // Wait for the stop event to complete

            float milliseconds = 0;

            // Calculate elapsed time
            cudaEventElapsedTime(&milliseconds, start, end);
            totalMilliseconds += milliseconds;
        }

        avgMilliseconds = totalMilliseconds / numRuns;
        printf("N = %d: Average Time for kernel_1t1c over %d runs: %f ms\n", N, numRuns, avgMilliseconds);

        // Wait for GPU to finish before accessing on host
        cudaDeviceSynchronize();

        // Copy result back to host
        cudaMemcpy(A, d_A, size, cudaMemcpyDeviceToHost);

        // Print results or check output
        printf("First 10 elements for N = %d:\n", N);
        for (int i = 0; i < 10 && i < N; i++) { // Example of printing first 10 elements
            printf("%f ", A[i]);
        }
        printf("\n");

        // Cleanup
        cudaEventDestroy(start);
        cudaEventDestroy(end);
        cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
        free(A); free(B); free(C);
    }

    return 0;
}



N = 4: Average Time for kernel_1t1c over 10 runs: 0.029888 ms
First 10 elements for N = 4:
124.835159 101.332863 77.915482 113.125603 
N = 16: Average Time for kernel_1t1c over 10 runs: 0.011264 ms
First 10 elements for N = 16:
74.517647 98.296608 18.021336 99.150787 143.830017 130.534576 104.634445 129.568481 97.333878 128.621811 
N = 64: Average Time for kernel_1t1c over 10 runs: 0.029440 ms
First 10 elements for N = 64:
51.953617 89.961128 61.007092 38.918518 170.510712 138.459351 109.247101 99.387833 164.447388 99.482315 
N = 256: Average Time for kernel_1t1c over 10 runs: 0.100413 ms
First 10 elements for N = 256:
40.772362 65.722710 128.192261 50.025650 61.464783 177.513412 120.943428 73.261261 154.609848 95.283554 
N = 1024: Average Time for kernel_1t1c over 10 runs: 1.130259 ms
First 10 elements for N = 1024:
136.829529 122.516586 98.160301 84.390724 27.842970 93.901207 71.524582 172.090012 108.210281 102.350830 
N = 4096: Average Time for kernel_1t1c over 10 runs: 15.772429 ms

# Analysis of each kernel

## `kernel_1t1c` (One Thread per Column)

Here, each thread is responsible for processing all the elements in a single column of matrices $B$ and $C$, with the results stored in the corresponding column of matrix $A$. The thread iterates over all rows for its assigned column to perform the addition.

### Key characteristics
* **Column-based focus**: Suited for operations that are inherently columnar in nature, potentially aligning better with subsequent column-based processing steps.
* **Reduced thread utilization**: Similar to `kernel_1t1e`, this approach activates fewer threads, which might be more efficient for specific GPU architectures or smaller matrices.

### Launch configuration

`kernel_1t1c`'s configuration typically involves one thread per column, which might lead to using a different number of threads per block, compared to `kerne_1t1r`.

* **Threads per block**: Commonly set as one thread per block if each thread handles an entire column.
* **Blocks per grid**. Set to match the number of columns in the matrix, ensuring that each column is processed by one thread.

### Memory access pattern

This pattern is also strided but in a transposed manner to `kernel_1t1r`.

* **Non-Coalesced Access**: Threads access matrix elements per column, which leads to a strided access pattern similar to `kernel_1t1r`, resulting in non-coalesced accesses.
* **Cache utilization**: Similar poor cache utilization due to the vertical strided pattern, potentially increasing memory latency and reducing throughput.

# Summary

Both `kernel_1t1r` and `kernel_1t1c` provide specific benefits for particular problem domains but tend to be less efficient than `kernel_1t1e` in terms of memory access and overall GPU resource utilization. These kernels might be preferred when subsequent operations in the computation pipeline benefit from having an entire row of column processed by a single thread, or when experimenting with memory traffic patterns on specific GPU architectures.

## Pros and cons

### `kernel_1t1e`

* **Pro**: Coalesced memory access. If aligned properly with memory boundaries, this kernel allows for coalesced memory accesses, where consecutive threads access consecutive memory addresses, leading to optimal use of the memory bandwidth.
* **Pro**: Load balancing. Every thread does exactly the same amount of work, which maximizes parallel efficiency and minimizes idle time for compute units.
* **Cons**: Potential for overhead. If the matrix is very large (which I did not show in this paper), the great number of threads can cause extra overhead due to thread management.

### `kernel_1t1r`

* **Pro**: Simple indexing. This method simplifies indexing logic, as each thread handles a complete row, which made it easier for me to manage the row additions.
* **Pro**: Reduced number of threads. Fewer threads felt like it was less of an overhead given my test data, which were essentially just small matrices.
* **Con**: Non-coalesced Memory Address. Threads access memory in a strided pattern, which can result in poor utilization of the memory bus and lower performance.
* **Con**: Load imbalance. In matrices where the number of columsn is not a power of two, or does not match well with the number of CUDA cores, some threads might be underutilized (CITE T4!)

### `kernel_1t1c`

* **Pro**: Suitable for column-based operations. It can be efficient if the subsequent operations or algorithms to be used are column-based, aligning better with certain linear algebra operations.
* **Cons**: Poor memory access pattern. Similar to `kernel_1t1r`, this approach suffers from strided memory access, which is inefficient on GPUs due to non-coalesced accesses.
* **Cons**: Load Imbalance. Like the row-wise kernel, the distribution of work might not be optimal, especially in wide or very narrow matrices.
